In [39]:
!pip install transformers
!pip install accelerate
!pip install pandas


In [42]:
import pandas as pd

df = pd.read_csv("spam.csv", encoding='latin-1')
print(df.columns)
# Then modify the original cell with the correct column names based on the output

Index(['text', 'target'], dtype='object')


In [44]:
import pandas as pd

df = pd.read_csv("spam.csv", encoding='utf-8')  # Adjust if needed
df = df.sample(n=50, random_state=42).reset_index(drop=True)  # Use 50 for faster testing
df.head()


,text,target
0,"Funny fact Nobody teaches volcanoes 2 erupt, t...",ham
1,I sent my scores to sophas and i had to do sec...,ham
2,We know someone who you know that fancies you....,spam
3,Only if you promise your getting out as SOON a...,ham
4,Congratulations ur awarded either å£500 of CD ...,spam


In [45]:
from transformers import pipeline

classifier = pipeline("text2text-generation", model="google/flan-t5-base", max_length=20)


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [46]:
def classify_email(text):
    prompt = f'Classify this email as either "spam" or "ham": "{text}"\nOnly reply with the label.'
    result = classifier(prompt)[0]['generated_text']
    return result.strip().lower()


In [47]:
from tqdm import tqdm

predictions = []

for email in tqdm(df['text']):
    label = classify_email(email)
    predictions.append(label)

df['llm_prediction'] = predictions


100%|██████████| 50/50 [00:51<00:00,  1.03s/it]


In [48]:
from sklearn.metrics import classification_report

# Normalize target labels
df['target'] = df['target'].str.lower()
print(classification_report(df['target'], df['llm_prediction']))


              precision    recall  f1-score   support

         ham       0.00      0.00      0.00        43
        spam       0.14      1.00      0.25         7

    accuracy                           0.14        50
   macro avg       0.07      0.50      0.12        50
weighted avg       0.02      0.14      0.03        50



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
